# Task 3 - Point Cloud Scenes

Visualizing large-scale point cloud scenes is important when qualitatively evaluating predictions from a machine learning model or exploring what is happening in an outdoor scene. To this end, using Python and Open3D (http://www.open3d.org/docs/release/getting_started.html), implement a visualization of a LiDAR point cloud sequence from the SemanticKITTI dataset. Your implementation should 1) allow you to navigate forward and backward in time using the left and right arrow keys, and 2) visualize the semantic labels for each point. An example visualization is shown in the following video (semantic labels on the left): https://www.youtube.com/watch?v=VRZGAK3XpX0.

In [62]:
import os
import open3d as o3d
import numpy as np

# Helper Functions

In [63]:
color_map = {
   0 : [0, 0, 0],
  1 : [0, 0, 255],
  10: [245, 150, 100],
  11: [245, 230, 100],
  13: [250, 80, 100],
  15: [150, 60, 30],
  16: [255, 0, 0],
  18: [180, 30, 80],
  20: [255, 0, 0],
  30: [30, 30, 255],
  31: [200, 40, 255],
  32: [90, 30, 150],
  40: [255, 0, 255],
  44: [255, 150, 255],
  48: [75, 0, 75],
  49: [75, 0, 175],
  50: [0, 200, 255],
  51: [50, 120, 255],
  52: [0, 150, 255],
  60: [170, 255, 150],
  70: [0, 175, 0],
  71: [0, 60, 135],
  72: [80, 240, 150],
  80: [150, 240, 255],
  81: [0, 0, 255],
  99: [255, 255, 50],
  252: [245, 150, 100],
  256: [255, 0, 0],
  253: [200, 40, 255],
  254: [30, 30, 255],
  255: [90, 30, 150],
  257: [250, 80, 100],
  258: [180, 30, 80],
  259: [255, 0, 0]
}

def get_path(task_number, *args):
    notebook_path = os.path.abspath(f"Task_{task_number}.ipynb")
    return os.path.join(os.path.dirname(notebook_path), *args)

def label_to_color(label: np.uint32):
    global color_map
    
    return np.array(color_map[label])

def np_to_voxel_grid(grid: np.ndarray, labels: np.ndarray):
    # Create new voxel grid object and set voxel_size to some value
    # --> otherwise it will default to 0 and the grid will be invisible
    voxel_grid = o3d.geometry.VoxelGrid()
    voxel_grid.voxel_size = 1
    
    # Iterate over numpy grid
    for z in range(grid.shape[2]):
        for y in range(grid.shape[1]):
            for x in range(grid.shape[0]):
                if grid[x, y, z] == 0:
                    continue
                
                # Create a voxel object
                voxel = o3d.geometry.Voxel()
                
                # Set the color depending on label
                voxel.color = label_to_color(labels[x][y][z])
                
                # Set position of voxel
                voxel.grid_index = np.array([x, y, z])
                
                # Add voxel object to grid
                voxel_grid.add_voxel(voxel)
                
    return voxel_grid

def load_voxel_grids(save_dir):
    voxel_grids = []
    
    for filename in os.listdir(save_dir):
        pcd = o3d.io.read_point_cloud(os.path.join(save_dir, filename))
        voxel_grids.append(pcd)
    
    return voxel_grids

def unpack(compressed):
  ''' given a bit encoded voxel grid, make a normal voxel grid out of it.  '''
  uncompressed = np.zeros(compressed.shape[0] * 8, dtype=np.uint8)
  uncompressed[::8] = compressed[:] >> 7 & 1
  uncompressed[1::8] = compressed[:] >> 6 & 1
  uncompressed[2::8] = compressed[:] >> 5 & 1
  uncompressed[3::8] = compressed[:] >> 4 & 1
  uncompressed[4::8] = compressed[:] >> 3 & 1
  uncompressed[5::8] = compressed[:] >> 2 & 1
  uncompressed[6::8] = compressed[:] >> 1 & 1
  uncompressed[7::8] = compressed[:] & 1

  return uncompressed


SPLIT_SEQUENCES = {
    "train": ["00", "01", "02", "03", "04", "05", "06", "07", "09", "10"],
    "valid": ["08"],
    "test": ["11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21"]
}

SPLIT_FILES = {
    "train": [".bin", ".label", ".invalid", ".occluded"],
    "valid": [".bin", ".label", ".invalid", ".occluded"],
    "test": [".bin"]
}

EXT_TO_NAME = {".bin": "input", ".label": "label", ".invalid": "invalid", ".occluded": "occluded"}

VOXEL_DIMS = (256, 256, 32)


class SSCDataset:
  def __init__(self, directory, split="train"):
    """ Load data from given dataset directory. """

    self.files = {}
    self.filenames = []

    for ext in SPLIT_FILES[split]:
      self.files[EXT_TO_NAME[ext]] = []

    for sequence in SPLIT_SEQUENCES[split]:
      complete_path = os.path.join(directory, "sequences", sequence, "voxels")
      if not os.path.exists(complete_path): raise RuntimeError("Voxel directory missing: " + complete_path)

      files = os.listdir(complete_path)
      for ext in SPLIT_FILES[split]:
        data = sorted([os.path.join(complete_path, f) for f in files if f.endswith(ext)])
        if len(data) == 0: raise RuntimeError("Missing data for " + EXT_TO_NAME[ext])
        self.files[EXT_TO_NAME[ext]].extend(data)

      # this information is handy for saving the data later, since you need to provide sequences/XX/predictions/000000.label:
      self.filenames.extend(
          sorted([(sequence, os.path.splitext(f)[0]) for f in files if f.endswith(SPLIT_FILES[split][0])]))

    self.num_files = len(self.filenames)

    # sanity check:
    for k, v in self.files.items():
      print(k, len(v))
      assert (len(v) == self.num_files)

  def __len__(self):
    return self.num_files

  def __getitem__(self, t):
    """ fill dictionary with available data for given index . """
    collection = {}

    # read raw data and unpack (if necessary)
    for typ in self.files.keys():
      scan_data = None
      if typ == "label":
        scan_data = np.fromfile(self.files[typ][t], dtype=np.uint16)
      else:
        scan_data = unpack(np.fromfile(self.files[typ][t], dtype=np.uint8))

      # turn in actual voxel grid representation.
      collection[typ] = scan_data.reshape(VOXEL_DIMS)

    return self.filenames[t], collection

# Generate Voxel Grids From Dataset

### Read from the dataset and create Dataset Object which stores voxel positions and labels

In [67]:
SAVE_DIR = get_path(3, "save", "segmentation")
DATASET_DIR = get_path(3, "data", "dataset")

dataset = SSCDataset(DATASET_DIR)
print("# files: {}".format(len(dataset)))

voxel_grids = []

for i in range(len(dataset)):
  (seq, filename), data = dataset[i]
  voxel_grid = data["input"]
  
  save_file_name = f"{seq}_{filename}.ply"
  save_file_path = os.path.join(SAVE_DIR, save_file_name)
  
  # check if file already exists
  if os.path.isfile(save_file_path):
    print("Voxel grid for entry " + str(i) + " (" + save_file_name + ") already exists. Skipping...")
    continue

  # Convert to Open3D voxel grid
  print("Generating Voxel Grid for entry " + str(i) + " (" + save_file_name + ")")
  voxel_grid = np_to_voxel_grid(voxel_grid, data["label"])
  voxel_grids.append(voxel_grid)
  
  # Save voxel grid to file
  o3d.io.write_voxel_grid(save_file_path, voxel_grid)

input 3834
label 3834
invalid 3834
occluded 3834
# files: 3834
Voxel grid for entry 0 (00_000000.ply) already exists. Skipping...
Voxel grid for entry 1 (00_000005.ply) already exists. Skipping...
Voxel grid for entry 2 (00_000010.ply) already exists. Skipping...
Voxel grid for entry 3 (00_000015.ply) already exists. Skipping...
Voxel grid for entry 4 (00_000020.ply) already exists. Skipping...
Voxel grid for entry 5 (00_000025.ply) already exists. Skipping...
Voxel grid for entry 6 (00_000030.ply) already exists. Skipping...
Voxel grid for entry 7 (00_000035.ply) already exists. Skipping...
Voxel grid for entry 8 (00_000040.ply) already exists. Skipping...
Voxel grid for entry 9 (00_000045.ply) already exists. Skipping...
Voxel grid for entry 10 (00_000050.ply) already exists. Skipping...
Voxel grid for entry 11 (00_000055.ply) already exists. Skipping...
Voxel grid for entry 12 (00_000060.ply) already exists. Skipping...
Voxel grid for entry 13 (00_000065.ply) already exists. Skippin

KeyboardInterrupt: 

# Run Open3D Visualization

In [68]:
# Load all saved voxel grids
SAVE_DIR = get_path(3, "save", "segmentation")
voxel_grids = load_voxel_grids(SAVE_DIR)

# Create a visualizer and window
vis = o3d.visualization.VisualizerWithKeyCallback()
vis.create_window()

# Add the first point cloud to the visualizer
vis.add_geometry(voxel_grids[0])

# This index will keep track of the current point cloud
idx = 0

# Define the functions to be called when the left or right arrow key is pressed
def left_key_action(vis):
    global idx
    idx = (idx - 1) % len(voxel_grids)
    vis.clear_geometries()
    vis.add_geometry(voxel_grids[idx])

def right_key_action(vis):
    global idx
    idx = (idx + 1) % len(voxel_grids)
    vis.clear_geometries()
    vis.add_geometry(voxel_grids[idx])

# Bind the functions to the left and right arrow keys
vis.register_key_callback(262, right_key_action)  # 262 is the key code for the right arrow key
vis.register_key_callback(263, left_key_action)  # 263 is the key code for the left arrow key

# Start the visualization
vis.run()
vis.destroy_window()